In [1]:
import os
import sys

from RFEM.enums import LineSupportType, SurfaceGeometry, SurfaceLoadDirection, SurfaceLoadDistribution, ActionCategoryType
from RFEM.initModel import Calculate_all, Model
from RFEM.BasicObjects.node import Node
from RFEM.BasicObjects.line import Line
from RFEM.BasicObjects.material import Material
from RFEM.BasicObjects.thickness import Thickness
from RFEM.BasicObjects.surface import Surface
from RFEM.TypesForLines.lineSupport import LineSupport
from RFEM.TypesForNodes.nodalSupport import NodalSupport
from RFEM.Results.resultTables import GetMaxValue, ResultTables
from RFEM.Loads.surfaceLoad import SurfaceLoad
from RFEM.LoadCasesAndCombinations.loadCase import LoadCase
from RFEM.ImportExport.exports import ExportTo

Model(True, 'ChatGPT_Tank.rf6')
Model.clientModel.service.begin_modification()

Connecting to server...


# Allgemeiner PROMPT

#### Ich möchte einen zylindrischen Stahldruckbehälter mit einem Durchmesser von 2 m, einer Höhe von 5 m und einem oberen Deckel unter Schneelast auf der Oberseite erstellen. Die untere Kreislinie ist gelagert. Die Dicke des Deckels beträgt 10 mm und die des Zylindermantels 4 mm. Die Schneelast beträgt 0.25 kN/m².


# MATERIAL

In [2]:
Material(1, "S235")

# GEOMETRY

In [3]:
# Define the bottom circle (base of the cylinder)
Node(1, 0, 0, 0)  # Center of the bottom circle
Node(2, 1, 0, 0)  # Point on the circumference of the bottom circle

# Define the top circle (top of the cylinder)
Node(3, 0, 0, -5)  # Center of the top circle (5m height in negative Z direction)
Node(4, 1, 0, -5)  # Point on the circumference of the top circle

# Define the lines for the bottom and top circles
Line.Circle(1, [0, 0, 0], 1, [0, 0, 1])  # Bottom circle with radius 1m
Line.Circle(2, [0, 0, -5], 1, [0, 0, 1])  # Top circle with radius 1m

# Define the vertical lines (cylinder sides)
Line(3, '2 4')  # Line from bottom circumference to top circumference

# Define the line support at the bottom circle
LineSupport(1, '1', LineSupportType.HINGED)

# CROSSSECTION

In [4]:
# Define the thickness for the top cover
Thickness(1, '10 mm', 1, 0.010)

# Define the thickness for the cylindrical shell
Thickness(2, '4 mm', 1, 0.004)

# Define the top cover as a flat plane surface
Surface.Standard(1, SurfaceGeometry.GEOMETRY_PLANE, None, '2', 1)

# Define the cylindrical shell as a surface of revolution
Surface.Standard(2, SurfaceGeometry.GEOMETRY_ROTATED, [360, [0, 0, 0], [0, 0, 1], 3], '3', 2)

# LOADS

In [5]:
# Define the load case without self-weight
LoadCase(1, 'Snow Load', self_weight=[False])

# Define the snow load on the top cover
SurfaceLoad.Force(
    no=1,
    load_case_no=1,
    surface_no='1',
    load_direction=SurfaceLoadDirection.LOAD_DIRECTION_GLOBAL_Z_OR_USER_DEFINED_W_TRUE,
    load_distribution=SurfaceLoadDistribution.LOAD_DISTRIBUTION_UNIFORM,
    load_parameter=[0.25 * 1000]
)

# Define the load case for self-weight with double effect in Z direction
LoadCase(2, 'Self-weight', self_weight=[True, 0.0, 0.0, 2.0])

# SOLVER AUFRUFEN

In [6]:
# ---- Solver Sec ----
Model.clientModel.service.finish_modification()

Calculate_all()

# ERGEBNISSE

In [7]:
# ---- Results Sec ----
global maxStress
maxStressinit = ResultTables.SurfacesEquivalentStressesMises()
maxStress = GetMaxValue(maxStressinit, 'equivalent_stresses_sigma_eqv_max') / 1000000

maxStress

9.153163